## Load configurations

In [1]:
from cfgs.content_classification_v2 import get_cfg

cfgs = get_cfg()
print('\n', cfgs, '\n')


 CLASSES: [{'Genre': {0: 'Comedy', 1: 'Action', 2: 'Documentary', 3: 'Crime', 4: 'Animation', 5: 'Horror'}, 'Rating': {0: 'value'}, 'Year': {0: 'value'}}]
HYPERPARAMS:
  BATCH_SIZE_TR: 64
  BATCH_SIZE_VAL: 64
  EPOCHS: 30
  LR: 0.0001
  TEST_SPLIT: 0.9
  VAL_SPLIT: 0.9
IMAGE:
  RESOLUTION: (200, 150, 3)
MODEL:
  BACKBONE: InceptionV3
  STRUCTURE: [{0: {'type': 'Genre', 'outNeurons': 6, 'outActivation': 'sigmoid', 'loss': 'categorical_crossentropy', 'weight': 1, 'metric': 'accuracy'}, 1: {'type': 'Rating', 'outNeurons': 1, 'outActivation': 'linear', 'loss': 'mse', 'weight': 1, 'metric': 'mae'}, 2: {'type': 'Year', 'outNeurons': 1, 'outActivation': 'linear', 'loss': 'mse', 'weight': 1, 'metric': 'mae'}}] 



In [2]:
dataset_dict = cfgs.CLASSES[0]
dataset_dict['Genre_alias'] = dict((g, i) for i, g in dataset_dict['Genre'].items())

## Load data and initialize generator

In [3]:
import pandas as pd
from data_processing.data_generator import DataGenerator
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_pickle("/home/robotics/content-classification/data_processing/balanced_data.pkl")

In [5]:
data_generator = DataGenerator(df, dataset_dict, cfgs.HYPERPARAMS.TEST_SPLIT, cfgs.HYPERPARAMS.VAL_SPLIT, cfgs.IMAGE.RESOLUTION)
data_generator.df.head()

,Index,Name,Year,Genre,Rating,Poster_path
0,tt0077838,The Last Waltz,1978,2,8.2,/home/robotics/Documents/data/Posters/1978/tt0...
1,tt0218080,Agent Red,2000,1,3.5,/home/robotics/Documents/data/Posters/2000/tt0...
2,tt0378889,"Tom, Tom, the Piper's Son",1969,0,6.2,/home/robotics/Documents/data/Posters/1969/tt0...
3,tt1183696,Second Skin,2009,2,6.6,/home/robotics/Documents/data/Posters/2009/tt1...
4,tt0068549,Essene,1972,2,7.1,/home/robotics/Documents/data/Posters/1972/tt0...


In [6]:
train_idx, valid_idx, test_idx = data_generator.split_dataset() 

## Build and compile model

In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from model.architecture import MultiOutputModel

In [8]:
structure = cfgs.MODEL.STRUCTURE[0]
content_model = MultiOutputModel(*cfgs.IMAGE.RESOLUTION, structure)
content_model.build_model("InceptionV3")
# keras.utils.plot_model(content_model.model, show_shapes=True)

In [9]:
# opt = SGD()
opt = Adam(lr=cfgs.HYPERPARAMS.LR, decay=cfgs.HYPERPARAMS.LR/cfgs.HYPERPARAMS.EPOCHS)
content_model.compile_model(opt)
content_model.model.summary()

Model: "content_classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 99, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 99, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 99, 74, 32)   0           batch_normalization[0][0]        
_____________________________________________________________________________

## Train Model

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.backend import clear_session
import tensorflow as tf

In [11]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

# GPU check - CUDA 11.1 with cuDNN 8.05 on 460.39 - GTX 1060 6GB
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU'))

clear_session()

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [12]:
tr_generator = data_generator.generate_images(train_idx, cfgs.HYPERPARAMS.BATCH_SIZE_TR, True)
val_generator = data_generator.generate_images(valid_idx, cfgs.HYPERPARAMS.BATCH_SIZE_VAL, True)

In [13]:
import datetime

%load_ext tensorboard
log_dir = log_dir = "./logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

callbacks = [keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)]
# keras.callbacks.ModelCheckpoint('./checkpoints', monitor='val_loss', verbose=0, 
#                                              save_best_only=False, save_weights_only=False, 
#                                              mode='auto', save_freq='epoch')

In [14]:
%%time
history = content_model.model.fit(tr_generator, 
                    steps_per_epoch=len(train_idx)//cfgs.HYPERPARAMS.BATCH_SIZE_TR,
                    epochs=cfgs.HYPERPARAMS.EPOCHS,
                    callbacks=callbacks,
                    validation_data=val_generator,
                    validation_steps=len(valid_idx)//cfgs.HYPERPARAMS.BATCH_SIZE_VAL)

Epoch 1/30
151/151 [==============================] - 34s 154ms/step - loss: 10.4419 - Genre_loss: 2.8386 - Rating_loss: 3.4644 - Year_loss: 4.1389 - Genre_accuracy: 0.1865 - Rating_mae: 1.4609 - Year_mae: 1.6432 - val_loss: 3.8404 - val_Genre_loss: 1.6752 - val_Rating_loss: 0.2519 - val_Year_loss: 1.9132 - val_Genre_accuracy: 0.2783 - val_Rating_mae: 0.4130 - val_Year_mae: 1.3596
Epoch 2/30
151/151 [==============================] - 21s 141ms/step - loss: 8.0842 - Genre_loss: 2.6250 - Rating_loss: 2.5486 - Year_loss: 2.9106 - Genre_accuracy: 0.2043 - Rating_mae: 1.2571 - Year_mae: 1.3643 - val_loss: 3.0722 - val_Genre_loss: 1.6113 - val_Rating_loss: 0.2206 - val_Year_loss: 1.2404 - val_Genre_accuracy: 0.3135 - val_Rating_mae: 0.3866 - val_Year_mae: 1.0858
Epoch 3/30
151/151 [==============================] - 22s 145ms/step - loss: 6.7598 - Genre_loss: 2.4279 - Rating_loss: 2.0586 - Year_loss: 2.2733 - Genre_accuracy: 0.2270 - Rating_mae: 1.1226 - Year_mae: 1.1954 - val_loss: 2.5917 - 

151/151 [==============================] - 21s 141ms/step - loss: 1.8393 - Genre_loss: 1.4587 - Rating_loss: 0.1658 - Year_loss: 0.2149 - Genre_accuracy: 0.4340 - Rating_mae: 0.3202 - Year_mae: 0.3691 - val_loss: 1.4861 - val_Genre_loss: 1.3994 - val_Rating_loss: 0.0292 - val_Year_loss: 0.0575 - val_Genre_accuracy: 0.4668 - val_Rating_mae: 0.1427 - val_Year_mae: 0.2312
Epoch 23/30
151/151 [==============================] - 21s 142ms/step - loss: 1.7812 - Genre_loss: 1.4373 - Rating_loss: 0.1451 - Year_loss: 0.1989 - Genre_accuracy: 0.4375 - Rating_mae: 0.2999 - Year_mae: 0.3532 - val_loss: 1.4904 - val_Genre_loss: 1.4053 - val_Rating_loss: 0.0271 - val_Year_loss: 0.0580 - val_Genre_accuracy: 0.4658 - val_Rating_mae: 0.1375 - val_Year_mae: 0.2328
Epoch 24/30
151/151 [==============================] - 21s 141ms/step - loss: 1.6890 - Genre_loss: 1.3920 - Rating_loss: 0.1241 - Year_loss: 0.1728 - Genre_accuracy: 0.4635 - Rating_mae: 0.2769 - Year_mae: 0.3290 - val_loss: 1.4848 - val_Genre_

## TensorBoard

In [15]:
%tensorboard --logdir logs

In [21]:
content_model.model.save("Model_v5")

INFO:tensorflow:Assets written to: Model_v5/assets


## Test set

In [16]:
import numpy as np

In [17]:
t_gen = data_generator.generate_images(test_idx, cfgs.HYPERPARAMS.BATCH_SIZE_TEST, False)
genre_inf, rating_inf, year_inf = content_model.model.predict(t_gen, 
                                                           steps=len(test_idx)//cfgs.HYPERPARAMS.BATCH_SIZE_TEST)

genres_pred = genre_pred.argmax(axis=-1)
ratings_pred = rating_pred * data_generator.max_rating
years_pred = year_pred * data_generator.max_year


* * * * * Image could not be loaded - skipping * * * * *


In [18]:
genres_true, ratings_true, years_true = generate_truth(self, test_idx, cfgs.HYPERPARAMS.BATCH_SIZE_TEST)


* * * * * Image could not be loaded - skipping * * * * *


In [19]:
from sklearn.metrics import classification_report

In [20]:
cr_genre = classification_report(genres_true, genres_pred, target_names=dataset_dict['Genre_alias'].keys())
print(cr_genre)

              precision    recall  f1-score   support

      Comedy       0.46      0.34      0.39       187
      Action       0.36      0.38      0.37       189
 Documentary       0.43      0.44      0.43       208
       Crime       0.37      0.41      0.39       186
   Animation       0.67      0.62      0.65       192
      Horror       0.43      0.51      0.47       190

    accuracy                           0.45      1152
   macro avg       0.46      0.45      0.45      1152
weighted avg       0.46      0.45      0.45      1152



In [23]:
from sklearn.metrics import r2_score
print('R2 score for ratings: ', r2_score(ratings_true, ratings_pred))

R2 score for ratings:  -0.240832386616592
